# Program Project 3: Sequence Randomization

12/14/2025
Tiffany Lin

## Project Question

- Randomize a sequence preserving its k-mer word content (i.e., 1 letter, 2-letter, 3-letter, ...). This may be solved simply using sampling with or without replacement, but the k-mer content of the randomized sequence will not exactly match the original sequence. A more complicated approach is to provide an exact solution using Euler’s method.
- input and output in FastA format. Your program should work for any alphanumeric letter sequence, but especially DNA and protein alphabets.
- k-mer should be selectable in range 1 to 6 (or more)
- Additional output should show original and randomized k-mer word content
- Bonus: 20 pts. Implement both a sampling solution and an exact solution (all k-mer counts exactly
the same as in the original sequence) using Euler’s algorithm.

## Introduction
This project implements two ways to randomize a sequence:
1. **Sampling randomization (approximate)**: shuffle characters in the sequence.
    - Preserves 1-mer composition exactly (same letters), but generally does not preserve k-mers for k ≥ 2.
2. **Exact k-mer preserving randomization (Euler / De Bruijn graph)**: build a **De Bruijn graph** from k-mers and **generate an Eulerian trail/circuit**.
    - Guarantees all k-mer counts match **exactly for k ≥ 2** (same multiset of k-mers), while still producing a randomized sequence by randomizing edge order.

## Problem Solving Strategy

To preserve the k-mer content, I first think of the difficulty of the problem depends on k. For k = 1, preserving k-mers only requires maintaining the overall letter composition of the sequence, which can be achieved by randomly shuffling characters. However, for k ≥ 2, simple shuffling disrupts local sequence order and therefore changes k-mer counts.


For this reason, I implemented two methods. The first is a **sampling-based randomization** that shuffles characters and serves as a baseline, illustrating that higher-order k-mer structure is not preserved. The second is an **exact method based on a De Bruijn graph representation of overlapping k-mers**. In this graph, each k-mer corresponds to a directed edge, and *an Eulerian traversal uses every edge exactly once*, guaranteeing that all k-mer counts are preserved. Randomness is introduced by shuffling edge traversal order, producing different valid sequences with identical k-mer content.

## Methods
The program reads FASTA input using Biopython, and the randomized sequences can be written back to FASTA (code provided but optional for demonstration).

**Input**
- FASTA file containing one sequence (DNA, protein, or any alphanumeric alphabet).
> In this demonstration, I use a short coding sequence (`real_cds.fa`) from *E. coli* as an example input from pp1.
- **User selects k (integer, 1 ≤ k ≤ sequence length).**
  
**k-mer counting**

For a sequence of length n, the number of overlapping k-mers is *n-k+1*

> $ \text{count}[w] = \#\{\, i \mid \text{seq}[i:i+k] = w \,\} $
> 
> count[w] is how many times the k-mer w appears in the sequence.
>
> **Example**
>
> Sequence: `ATACCACC`, k = 4, w = `ACCA`
> count[`ACCA`] = 1


In [31]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from collections import Counter, defaultdict
import random
import os

In [32]:
# Load the input FASTA file
input_fasta = "/Users/rusher/Desktop/biol595_extra_credit/pp3/data/real_cds.fa"
record = SeqIO.read(input_fasta, "fasta")
seq = str(record.seq)

print("Input ID:", record.id)
print("Length:", len(seq))
print("First 60:", seq[:60])

Input ID: lcl|NC_000913.3_cds_NP_414542.1_1
Length: 66
First 60: ATGAAACGCATTAGCACCACCATTACCACCACCATCACCATTACCACAGGTAACGGTGCG


In [33]:
# Prompt user for k-mer size
user_input = input("Enter k-mer size (integer ≥ 1, e.g. 1–6 or more): ").strip()

if not user_input.isdigit():
    raise ValueError("k must be a positive integer.")

k = int(user_input)
if k < 1:
    raise ValueError("k must be ≥ 1.")
if k > len(seq):
    raise ValueError("k must be ≤ sequence length.")

### Method 1: Sampling Randomization
- Convert the sequence to a list of characters, shuffle, and rejoin.
- Compare original vs randomized k-mer count tables.

**Expectation:**
- For k = 1, this matches exactly.
- For k ≥ 2, it usually differs.

In [34]:
def count_kmers(seq, k):
    """
    This function counts how many times each k-mer appears in a sequence.
    """
    return Counter(seq[i:i+k] for i in range(len(seq)-k+1))

In [35]:
# Method 1: Sampling randomization by shuffling
def sampling_randomize(seq, seed=None):
    rng = random.Random(seed)
    s = list(seq) # Convert to list for shuffling
    rng.shuffle(s) # Randomly rearranges the order of characters in the list.
    return "".join(s)

In [36]:
orig = count_kmers(seq, k)
rand_sampling = sampling_randomize(seq, seed=1)
rand_sampling_counts = count_kmers(rand_sampling, k)

print("Sampling randomized (first 60):", rand_sampling[:60])
print("Exact k-mer match (sampling)?", orig == rand_sampling_counts)

Sampling randomized (first 60): CCTCCAAGGAAATTGAACATTAATCCACACCAGGGATGAACCCCAGCACATTGCGTATAC
Exact k-mer match (sampling)? False


---

### Method 2: Exact (Eulerian path on De Bruijn graph)
- Each k-mer contributes a directed edge from prefix (k−1)-mer to suffix (k−1)-mer.
  - Node: length (k−1) string
  - Edge: one observed k-mer occurrence
- Finding an Eulerian trail uses every edge exactly once, so it uses each k-mer exactly as many times as it appears in the original sequence.
- Reconstruct output sequence by walking the node path and appending the last character of each next node.
- Randomness is introduced by shuffling adjacency lists before traversal (different Eulerian paths → different outputs, same k-mer counts).

**Notes:**
- Exact method **requires k ≥ 2** (for k=1, Euler graph concept is unnecessary).

In [37]:
def build_debruijn_graph(seq, k):
    """
    This function builds a De Bruijn graph from the k-mers of a sequence.
    """
    adj = defaultdict(list) # adjacency list mapping from node to list of neighbors
    indeg = Counter() # count of incoming edges for each node
    outdeg = Counter() # count of outgoing edges for each node

    # loop over all k-mers in the sequence
    for i in range(len(seq) - k + 1):
        kmer = seq[i:i+k]
        u = kmer[:-1] # prefix (first k-1 chars)
        v = kmer[1:] # suffix (last k-1 chars)
        adj[u].append(v)
        outdeg[u] += 1
        indeg[v] += 1
        if v not in adj: # ensure all nodes appear in adjacency list
            adj[v] = adj[v]
    return adj, indeg, outdeg # return adjacency list and in/out degrees

In [38]:
def choose_start_node(adj, indeg, outdeg):
    """This function selects an appropriate starting node for finding an Eulerian path or circuit in the De Bruijn graph."""
    # start node for Euler trail if exists
    for node in adj.keys():
        # In a directed graph, an Eulerian path (not a cycle) starts at a node with exactly one more outgoing edge than incoming edges. 
        # If this node exist, it will be the start of the path.
        if outdeg[node] == indeg[node] + 1:
            return node
    # otherwise circuit: any node with edges
    for node in adj.keys():
        # If no Eulerian path start node exists, start at any node with outgoing edges (Eulerian circuit)
        if len(adj[node]) > 0:
            return node
    raise ValueError("No edges in graph (sequence too short for this k).") # If no edges exist (e.g., the sequence is too short for the chosen k)

In [39]:
def eulerian_path(adj, start):
    """This function finds an Eulerian path or circuit in a directed graph using Hierholzer's algorithm."""
    stack = [start]
    path = []

    # Traverse the graph
    while stack:
        v = stack[-1] # current node

        
        # If the current node has outgoing edges that haven't been used, take one.
        #pop() removes the edge so it cannot be reused, ensuring each edge is visited exactly once.
        if adj[v]:
            stack.append(adj[v].pop())

        # When a node has no remaining outgoing edges, it is finished. It is added to the final path, and the algorithm backtracks.
        else:
            path.append(stack.pop())
    path.reverse() # reverse to get correct order
    return path

In [40]:
def reconstruct_from_path(node_path):
    """This function reconstructs the final sequence from the Eulerian node path produced by the graph traversal."""
    seq_out = [node_path[0]]
    for node in node_path[1:]:
        seq_out.append(node[-1]) # Each consecutive node overlaps the previous one by k−2 characters. only the last character of each node is new and needs to be appended.
    return "".join(seq_out) # join the list into a single string

In [41]:
def exact_kmer_preserving_randomize(seq, k, seed=None):
    """The function builds a graph from k-mers, randomly chooses how to traverse it, walks through every k-mer exactly once, 
    and then reconstructs a new sequence that has exactly the same k-mer content as the original."""
    if k < 2:
        raise ValueError("Exact Euler method requires k >= 2.")
    adj, indeg, outdeg = build_debruijn_graph(seq, k)

    # Randomize the order of edges in the adjacency list
    rng = random.Random(seed)
    for u in adj:
        rng.shuffle(adj[u])  # randomize traversal among edges

    # Choose starting node for Eulerian path
    start = choose_start_node(adj, indeg, outdeg)
    node_path = eulerian_path(adj, start)
    return reconstruct_from_path(node_path)

In [42]:
if k >= 2:
    rand_exact = exact_kmer_preserving_randomize(seq, k, seed=1)
    rand_exact_counts = count_kmers(rand_exact, k)

    print("Exact randomized (first 60):", rand_exact[:60])
    print("Exact k-mer match (Euler)?", orig == rand_exact_counts)
else:
    rand_exact = None
    print("k=1: exact method not needed; sampling shuffle already preserves 1-mers exactly.")

Exact randomized (first 60): ATGAAACGCATTACCACCATTAGCACCACCACCATCACCATTACCACAGGTAACGGTGCG
Exact k-mer match (Euler)? True


This code compares k-mer distributions across different sequences:
1. **Original sequence**
    - Displays the most frequent k-mers in the input sequence.
2. **Sampling randomized sequence**
    - Shows how k-mer frequencies change after simple shuffling, illustrating that higher-order k-mers are not preserved.
3. **Euler randomized sequence (exact method)**
    - Displays k-mer frequencies after exact randomization, confirming that the k-mer counts match the original sequence.


In [43]:
def show_top(counter, top=15):
    """This function displays the top k-mers from a given k-mer counter."""
    for kmer, c in counter.most_common(top):
        print(f"{kmer}\t{c}")

print("\nTop k-mers in original:")
show_top(orig)

print("\nTop k-mers in sampling randomized:")
show_top(rand_sampling_counts)

if rand_exact is not None:
    print("\nTop k-mers in Euler randomized:")
    show_top(rand_exact_counts)


Top k-mers in original:
CACCA	5
ACCAC	4
CATTA	3
CCACC	3
ACCAT	3
CCATT	2
ATTAC	2
TTACC	2
TACCA	2
ATGAA	1
TGAAA	1
GAAAC	1
AAACG	1
AACGC	1
ACGCA	1

Top k-mers in sampling randomized:
TGAAC	2
ACATT	2
CCTCC	1
CTCCA	1
TCCAA	1
CCAAG	1
CAAGG	1
AAGGA	1
AGGAA	1
GGAAA	1
GAAAT	1
AAATT	1
AATTG	1
ATTGA	1
TTGAA	1

Top k-mers in Euler randomized:
CACCA	5
ACCAC	4
CATTA	3
CCACC	3
ACCAT	3
ATTAC	2
TTACC	2
TACCA	2
CCATT	2
ATGAA	1
TGAAA	1
GAAAC	1
AAACG	1
AACGC	1
ACGCA	1


In [44]:
#out_dir = "/Users/rusher/Desktop/biol595_extra_credit/pp3/data"
#
## sampling output
#SeqIO.write(
#    SeqRecord(Seq(rand_sampling), id=f"randomized_sampling_k{k}", description="sampling_shuffle"),
#    os.path.join(out_dir, f"randomized_sampling_k{k}.fa"),
#    "fasta"
#)
#
## exact output (if exists)
#if rand_exact is not None:
#    SeqIO.write(
#        SeqRecord(Seq(rand_exact), id=f"randomized_exact_k{k}", description="euler_exact"),
#        os.path.join(out_dir, f"randomized_exact_k{k}.fa"),
#        "fasta"
#    )

## Results
I tested the program on the FASTA input sequence `real_cds.fa` (length 66 nt; ID lcl|NC_000913.3_cds_NP_414542.1_1) and selected **k = 5** for the demonstration. Using the sampling randomization method (character shuffling), the randomized sequence visibly differs from the original sequence and the k-mer composition does not match: the program reports `Exact k-mer match (sampling)? False`, and the most frequent 5-mers reported after shuffling differ from those in the original sequence. 

In contrast, using the Euler/De Bruijn exact randomization method, the program reports `Exact k-mer match (Euler)? True`, and **the top k-mers and their counts match the original k-mer frequency table**, confirming that all observed k-mers are preserved exactly. For readability, the program prints only the first 60 characters of each randomized sequence, while all k-mer counting and comparisons are computed using the full sequences.

## Discussion/Conclusion
This project shows that a simple shuffle based randomization is a useful baseline because it preserves 1-mer composition, but it does not preserve k-mer word content for k ≥ 2 since shuffling destroys local adjacency relationships between characters. The exact method solves this by converting the sequence’s k-mers into a De Bruijn graph where each k-mer corresponds to a directed edge between its *(k−1)-mer prefix* and *suffix*; an Eulerian traversal visits every edge exactly once, which guarantees that the output contains the same multiset of k-mers as the input. Randomness is introduced by shuffling adjacency lists before traversal, producing different valid randomized sequences while maintaining identical k-mer counts. **Therefore, the sampling approach provides approximate randomization, while the Eulerian approach meets the project requirement of exact k-mer preservation.**
